<a href="https://colab.research.google.com/github/chhatarshal/MachineLearningPractice/blob/dev/colab/001_Introduction_to_NLP_Fundamentals_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#https://github.com/chhatarshal/MachineLearningPractice/blob/main/tensorflow/helper/helper_functions.py

Lets get our helper function file using wget. We need this file to do some common task like unzip

In [35]:
!wget https://raw.githubusercontent.com/chhatarshal/MachineLearningPractice/main/tensorflow/helper/helper_functions.py

--2021-07-11 11:16:00--  https://raw.githubusercontent.com/chhatarshal/MachineLearningPractice/main/tensorflow/helper/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-07-11 11:16:00 (83.7 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



Lets now import some common function from helper functions file

In [36]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

Now we are going to get data for our NLP research and will use unzip_data function from common functions to unzip all text data

In [37]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2021-07-11 11:16:00--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-07-11 11:16:00 (102 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



Now we got data next important think we need to do is check the data visualize visualize visualize

In [38]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


I saw some NaN in keyword and location column


In [39]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) 
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [40]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


Lets see how many positive and negative sample we have

In [41]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

Here we see we have 4342 records says tweet are not disaster and rest 3271 tweets are disaster

In [42]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

We have 7613 train data and 3263 test data

Lets now visualize some random data

In [43]:
import random

In [44]:
random_index = random.randint(0, len(train_df))

In [45]:
print(random_index)

202


In [46]:
train_df.iloc[random_index]["text"]

'http://t.co/X5YEUYLT1X Twelve feared killed in Pakistani air ambulance helicopter crash http://t.co/2UgrMd1z1n'

In [47]:
for i in range(10):
  random_index = random.randint(0, len(train_df))
  train_df.iloc[random_index]["text"]
  text, target = train_df.iloc[random_index][["text", "target"]]
  print(text, '\n', target)
  print(f"\n\n")

ColnHarun: Breaking news! Unconfirmed! I just heard a loud bang nearby. in what appears to be a blast of wind from my neighbour's ass. 
 0



#Obama signed up to a deal that far from making the world a safer place http://t.co/E0luGBL6pb via @upi #Iran #Nuclear #IranNuclearDeal 
 1



Woke up so blown lol 
 0



The Dress Memes Have Officially Exploded On The Internet http://t.co/3drSmxw3cr 
 1



MEN CRUSH EVERY FUCKING DAY???????????????????????????? http://t.co/Fs4y1c9mNf 
 0



Setting Up An Emergency Fund In 3 Easy Steps: You never know when a surprise expense will pop up. So work up t... http://t.co/Iz17kLelZC 
 0



@omgbethersss @BethanyMota haha love this?? 
 0



2 Injured 1 missing in bridge collapse in central Mexico - Fox News Latino http://t.co/l0UnaFLy0Y 
 1



@CTAZtrophe31 Everything must be OK because she's listening to 'Sandstorm' now... 
 0



Bamenda Floods Kill Animals Birds - http://t.co/WnamtxlfMt http://t.co/6cOIDv11qV 
 1





We have to divide data between training and validation data and sklearn is best tool to do that

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation split
                                                                            random_state=42)

In [50]:

# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [51]:

# Check the first 10 samples 
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 


Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:

Tokenziation - direct mapping of token (a token could be a word or a character) to number
Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

In [52]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be?
                                    pad_to_max_tokens=True)

In [53]:

len(train_sentences[0].split())

7

In [54]:
train_sentences[0]

'@mogacola @zamtriossu i screamed after hitting tweet'

In [55]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [56]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [57]:
# Fit the text vectorizer instance to the training data using the adapt() method
text_vectorizer.adapt(train_sentences)

In [58]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [59]:
# Choose a random sentence from the training dataset and tokenize it
for i in range(10):
  random_sentence = random.choice(train_sentences)
  print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:")
  text_vectorizer([random_sentence])

Original text:
@danagould @WaynesterAtl I agree with background checks. I just think guns or weapons in general are the great equalizer.        

Vectorized version:
Original text:
With a sinking music video tv career Brooke Hogan should be THANKING her Dad for the free publicity...although I doubt it will help her.        

Vectorized version:
Original text:
@TR_jdavis Bruh you wanna fight I'm down meet me in the cage bro better find out who you're dealing with before you end up in a body bag        

Vectorized version:
Original text:
Fuck Sleeping With Sirens.        

Vectorized version:
Original text:
Canada: Hailstorm flash flooding slam Calgary knocks out power to 20k customers
http://t.co/SkY9EokgGB http://t.co/5IyZsDA6xB        

Vectorized version:
Original text:
@rachelcaine The weatherit needs to make it minds up. First snow tornadoes now would you say a heat wave?        

Vectorized version:
Original text:
@adriennetomah how did people survive like that?!        

Vectori

In [60]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # the most common words in the vocab
bottom_5_words = words_in_vocab[-5:] # the least common words in the vocab
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab: {bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [61]:
# how long is our vocab?
len(words_in_vocab)

10000

In [62]:
max_length,

(15,)

In [63]:

from tensorflow.keras import layers 

embedding = layers.Embedding(input_dim=max_vocab_length, # set the input shape
                             output_dim=128, # set the size of the embedding vector
                             embeddings_initializer="uniform", # default, initialize embedding vectors randomly
                             input_length=max_length # how long is each input
                             )

embedding

In [64]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation, aka tokenization first)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
'I may have gotten a little too exited over my (home made) lava lamp.':http://t.co/724Gq5ebqZ http://t.co/H01j9PIrIe        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0274994 ,  0.0059594 , -0.03029578, ...,  0.00358319,
         -0.03740345, -0.01458687],
        [ 0.01756116, -0.01778661,  0.03354332, ...,  0.0383457 ,
          0.01279375,  0.00856616],
        [ 0.02726544, -0.01134182,  0.02890546, ...,  0.04046388,
         -0.04066104,  0.0107677 ],
        ...,
        [ 0.00687023,  0.03294111,  0.04665304, ...,  0.01511109,
          0.03874833, -0.00713354],
        [ 0.03350553,  0.00618553, -0.00930693, ...,  0.00053268,
         -0.03723173, -0.04340067],
        [ 0.03350553,  0.00618553, -0.00930693, ...,  0.00053268,
         -0.03723173, -0.04340067]]], dtype=float32)>

In [65]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.0274994 ,  0.0059594 , -0.03029578,  0.03211072, -0.00406014,
        0.04711392,  0.04878405, -0.03943096,  0.02357394,  0.020131  ,
       -0.03134946,  0.0206232 , -0.02541631, -0.0197214 ,  0.01968742,
       -0.04837043, -0.01441659,  0.02491375,  0.02482093,  0.00058125,
        0.01812496,  0.03868731,  0.03622136,  0.02559414, -0.02354709,
       -0.0471715 , -0.04928783, -0.04203956,  0.03729324,  0.03827825,
       -0.01323409, -0.00480045,  0.043523  ,  0.0079351 , -0.03842805,
        0.01625805, -0.03069288, -0.0001277 , -0.02199288, -0.04994371,
        0.00351308, -0.01296034,  0.03398535, -0.02142787,  0.03435001,
       -0.01770376,  0.00441916, -0.03751186,  0.00760585,  0.01536205,
        0.00772304, -0.03990846,  0.01106299, -0.01030837, -0.03765128,
        0.01912138, -0.01931379,  0.0409385 ,  0.03440132, -0.0171173 ,
       -0.03714307,  0.01603789, -0.04513511,  0.01084025,  0.03006544,
       -0.041196

In [67]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
                    ("clf", MultinomialNB()) # model the text converted to numbers
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

CPU times: user 197 ms, sys: 3.04 ms, total: 200 ms
Wall time: 223 ms


In [68]:
# Let's evalaute our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [69]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [70]:
# Function to evalaute: accuracy, precision, recall, F1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  ----
  y_true = true labels in the form of a 1D array
  y_pred = predicted label in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall and f1-score between y_true and y_pred.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100 # get accuracy score in percentage value
  # Calculate model precision, recall and f1 score using "weighted" avergage
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  # Create a dictionary of model results
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [71]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

Model 1: A simple dense model (feed-forward neural network)

In [82]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers 
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_sentences,
                        train_labels, 
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210711-114753
Epoch 1/5
215/215 [==============================] - 4s 16ms/step - loss: 0.5362 - accuracy: 0.7761 - val_loss: 0.5002 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3480 - accuracy: 0.8847 - val_loss: 0.4590 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2748 - accuracy: 0.9041 - val_loss: 0.4601 - val_accuracy: 0.7861
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2320 - accuracy: 0.9190 - val_loss: 0.4751 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2004 - accuracy: 0.9304 - val_loss: 0.4926 - val_accuracy: 0.7861


In [83]:
# Evaluate model 1
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4926 - accuracy: 0.7861


[0.49255895614624023, 0.7860892415046692]

In [84]:
# Make predictions with model_1
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

array([[0.47187176],
       [0.81861365],
       [0.99817526],
       [0.1385951 ],
       [0.07626599],
       [0.9520124 ],
       [0.91672105],
       [0.9960978 ],
       [0.981508  ],
       [0.35804135]], dtype=float32)

In [85]:
# Convert model_1 pred probs from probabilities to prediction labels
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [86]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [87]:
# Evaluate model_1 with our evaluation function
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'f1': 0.7843264664117531,
 'precision': 0.787706626889381,
 'recall': 0.7860892388451444}

In [88]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [89]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [90]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_3 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [92]:
# Get the weight matrix of embedding layer
# (the weights are the numerical patterns between the text in the training dataset that the model has learned)
embed_weights = model_1.get_layer("embedding_1 ").get_weights()[0]
print(embed_weights.shape)

ValueError: ignored

In [99]:
embed_weights = model_1.get_weights()[2]

In [100]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [101]:
# Let's download the saved embeddings locally
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>